# Preprocessing Training logs

In [22]:
import os
import sys
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Any

class TrafficDataProcessor:
    def __init__(self, q_table_path: str):
        """Initialize with path to Q-table JSON file"""
        self.q_table_path = q_table_path
        self.q_table = self.load_q_table()
        
    def load_q_table(self) -> Dict:
        """Load and preprocess Q-table from JSON"""
        with open(self.q_table_path, 'r') as f:
            raw_table = json.load(f)
        return {eval(k): v for k, v in raw_table.items()}
    
    def get_dashboard_data(self) -> Dict[str, Any]:
        """Generate all data needed for dashboard"""
        return {
            'action_distribution': self.calculate_action_distribution(),
            'lane_metrics': self.analyze_lane_preferences(),
            'training_progress': self.generate_training_metrics(),
            'value_stats': self.analyze_q_values()
        }
    
    def calculate_action_distribution(self) -> List[Dict]:
        """Analyze preferred actions across all states"""
        actions = []
        for values in self.q_table.values():
            actions.append(np.argmax(values))
        
        counts = pd.Series(actions).value_counts()
        total = len(actions)
        
        return [
            {'action': 'Left', 'percentage': float(counts.get(0, 0)/total * 100)},
            {'action': 'Stay', 'percentage': float(counts.get(1, 0)/total * 100)},
            {'action': 'Right', 'percentage': float(counts.get(2, 0)/total * 100)}
        ]
    
    def analyze_lane_preferences(self) -> List[Dict]:
        """Analyze lane utilization and performance"""
        lane_stats = {i: {'count': 0, 'avg_value': 0} for i in range(1, 6)}
        
        for state, values in self.q_table.items():
            lane = state[1]
            if lane in lane_stats:
                lane_stats[lane]['count'] += 1
                lane_stats[lane]['avg_value'] += np.max(values)
        
        # Calculate averages and prepare return format
        return [
            {
                'lane': lane,
                'visits': stats['count'],
                'avg_value': stats['avg_value']/stats['count'] if stats['count'] > 0 else 0,
            }
            for lane, stats in lane_stats.items()
        ]
    
    def generate_training_metrics(self) -> Dict:
        """Generate metrics showing training progress"""
        states = list(self.q_table.keys())
        time_steps = len(states)
        
        # Generate episode rewards (estimated from Q-values)
        rewards = []
        rolling_rewards = []
        window = 50  # Rolling average window
        
        for i in range(time_steps):
            if i < len(states):
                reward = np.max(self.q_table[states[i]])
                rewards.append(reward)
                
                # Calculate rolling average
                if i >= window:
                    avg = np.mean(rewards[i-window:i])
                else:
                    avg = np.mean(rewards[:i+1])
                rolling_rewards.append(avg)
        
        return {
            'episodes': list(range(len(rewards))),
            'rewards': rewards,
            'rolling_avg': rolling_rewards
        }
    
    def analyze_q_values(self) -> Dict:
        """Statistical analysis of Q-values"""
        all_values = []
        for values in self.q_table.values():
            all_values.extend(values)
            
        return {
            'mean': float(np.mean(all_values)),
            'std': float(np.std(all_values)),
            'min': float(np.min(all_values)),
            'max': float(np.max(all_values)),
            'distribution': np.histogram(all_values, bins=20)[0].tolist()
        }

In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

class TrafficDashboard:
    def __init__(self, dashboard_data: Dict):
        self.data = dashboard_data
        
    def create_dashboard(self) -> go.Figure:
        # Create figure with subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Training Progress',
                'Action Distribution',
                'Lane Utilization',
                'Q-Value Distribution'
            )
        )
        
        # Add all subplots
        self._add_training_progress(fig, 1, 1)
        self._add_action_distribution(fig, 1, 2)
        self._add_lane_metrics(fig, 2, 1)
        self._add_value_distribution(fig, 2, 2)
        
        # Update overall layout
        fig.update_layout(
            height=800,
            showlegend=True,
            title_text="Traffic Simulation Q-Learning Analysis",
            template='plotly_white',
            font=dict(size=12)
        )
        
        return fig
    
    def _add_training_progress(self, fig: go.Figure, row: int, col: int):
        """Add training progress subplot with improved axes"""
        metrics = self.data['training_progress']
        
        # Add episode rewards scatter
        fig.add_trace(
            go.Scatter(
                x=metrics['episodes'],
                y=metrics['rewards'],
                mode='markers',
                name='Episode Rewards',
                marker=dict(
                    size=4,
                    opacity=0.5,
                    color='blue'
                ),
                hovertemplate="Episode: %{x}<br>Reward: %{y:.2f}<extra></extra>"
            ),
            row=row, col=col
        )
        
        # Add rolling average line
        fig.add_trace(
            go.Scatter(
                x=metrics['episodes'],
                y=metrics['rolling_avg'],
                mode='lines',
                name='Rolling Average (50 episodes)',
                line=dict(
                    color='red',
                    width=2
                ),
                hovertemplate="Episode: %{x}<br>Avg Reward: %{y:.2f}<extra></extra>"
            ),
            row=row, col=col
        )
        
        # Update axes for this subplot
        fig.update_xaxes(
            title_text="Training Episodes",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            gridcolor='lightgray',
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Cumulative Reward",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            gridcolor='lightgray',
            row=row, col=col
        )
        
        # Add axis annotations
        fig.add_annotation(
            text="Episode rewards include:<br>• Distance covered<br>• Lane change penalties (-5)<br>• Time penalties (-10)",
            xref="paper", yref="paper",
            x=0.02, y=0.98,
            showarrow=False,
            font=dict(size=10),
            align="left",
            bgcolor="rgba(255,255,255,0.8)",
            row=row, col=col
        )
    
    def _add_action_distribution(self, fig: go.Figure, row: int, col: int):
        """Add action distribution subplot with improved axes"""
        actions = self.data['action_distribution']
        
        fig.add_trace(
            go.Bar(
                x=[d['action'] for d in actions],
                y=[d['percentage'] for d in actions],
                name='Action Distribution',
                text=[f"{d['percentage']:.1f}%" for d in actions],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(
            title_text="Agent Actions",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Selection Frequency (%)",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            range=[0, 100],
            row=row, col=col
        )
    
    def _add_lane_metrics(self, fig: go.Figure, row: int, col: int):
        """Add lane utilization subplot with improved axes"""
        lanes = self.data['lane_metrics']
        
        # Bar chart for visits
        fig.add_trace(
            go.Bar(
                x=[d['lane'] for d in lanes],
                y=[d['visits'] for d in lanes],
                name='Lane Visits',
                text=[f"{d['visits']}" for d in lanes],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        # Line for average Q-values
        fig.add_trace(
            go.Scatter(
                x=[d['lane'] for d in lanes],
                y=[d['avg_value'] for d in lanes],
                name='Avg Q-Value',
                mode='lines+markers',
                line=dict(color='red'),
                yaxis='y2'
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(
            title_text="Lane Number",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Visit Count",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        # Add secondary y-axis for Q-values
        fig.update_yaxes(
            title_text="Average Q-Value",
            secondary_y=True,
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
    
    def _add_value_distribution(self, fig: go.Figure, row: int, col: int):
        """Add Q-value distribution subplot with improved axes"""
        values = self.data['value_stats']
        
        fig.add_trace(
            go.Histogram(
                x=values['distribution'],
                name='Q-Value Distribution',
                nbinsx=20,
                histnorm='percent'
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(
            title_text="Q-Value Range",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )
        
        fig.update_yaxes(
            title_text="Frequency (%)",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            row=row, col=col
        )



In [24]:
processor = TrafficDataProcessor('./TD_Task1/Weighted_model.json')
dashboard_data = processor.get_dashboard_data()
dashboard = TrafficDashboard(dashboard_data)
fig = dashboard.create_dashboard()

# Update figure layout for better visibility
fig.update_layout(
    title=dict(
        text="Weighted Model Training",
        x=0.5,
        y=0.95,
        xanchor='center',
        font=dict(size=20)
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255,255,255,0.8)"
    ),
    margin=dict(t=100, b=50, l=50, r=50))

fig.show()

In [25]:
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Any
from collections import defaultdict

class EvaluationDataProcessor:
    def __init__(self, eval_log_path: str):
        """Initialize with path to evaluation log JSON file"""
        self.eval_log_path = eval_log_path
        self.eval_data = self.load_eval_logs()
        
    def load_eval_logs(self) -> List[Dict]:
        """Load evaluation logs from JSON file"""
        eval_episodes = []
        with open(self.eval_log_path, 'r') as f:
            for line in f:
                eval_episodes.append(json.loads(line.strip()))
        return eval_episodes
    
    def get_evaluation_metrics(self) -> Dict[str, Any]:
        """Generate all evaluation metrics for dashboard"""
        return {
            'episode_metrics': self.calculate_episode_metrics(),
            'action_distribution': self.calculate_action_distribution(),
            'lane_transitions': self.analyze_lane_transitions(),
            'reward_analysis': self.analyze_rewards(),
            'state_coverage': self.analyze_state_coverage()
        }
    
    def calculate_episode_metrics(self) -> Dict:
        """Calculate per-episode metrics including total reward, steps, and completion rate"""
        episodes = []
        total_rewards = []
        num_steps = []
        
        for episode in self.eval_data:
            episodes.append(episode['Episode'])
            episode_reward = sum(step['Reward'] for step in episode['Timesteps'])
            total_rewards.append(episode_reward)
            num_steps.append(len(episode['Timesteps']))
            
        return {
            'episodes': episodes,
            'rewards': total_rewards,
            'steps': num_steps,
            'rolling_avg_reward': self._calculate_rolling_average(total_rewards, window=10),
            'rolling_avg_steps': self._calculate_rolling_average(num_steps, window=10)
        }
    
    def calculate_action_distribution(self) -> List[Dict]:
        """Analyze distribution of actions taken during evaluation"""
        action_counts = defaultdict(int)
        total_actions = 0
        
        for episode in self.eval_data:
            for step in episode['Timesteps']:
                action_counts[step['Action']] += 1
                total_actions += 1
        
        return [
            {'action': 'Left', 'percentage': float(action_counts[-1]/total_actions * 100)},
            {'action': 'Stay', 'percentage': float(action_counts[0]/total_actions * 100)},
            {'action': 'Right', 'percentage': float(action_counts[1]/total_actions * 100)}
        ]
    
    def analyze_lane_transitions(self) -> List[Dict]:
        """Analyze lane change patterns and lane utilization"""
        lane_visits = defaultdict(int)
        lane_rewards = defaultdict(list)
        
        for episode in self.eval_data:
            for step in episode['Timesteps']:
                current_lane = step['State'][1]
                lane_visits[current_lane] += 1
                lane_rewards[current_lane].append(step['Reward'])
        
        return [
            {
                'lane': lane,
                'visits': visits,
                'avg_reward': np.mean(lane_rewards[lane]),
                'std_reward': np.std(lane_rewards[lane])
            }
            for lane, visits in sorted(lane_visits.items())
        ]
    
    def analyze_rewards(self) -> Dict:
        """Detailed analysis of rewards during evaluation"""
        all_rewards = []
        for episode in self.eval_data:
            for step in episode['Timesteps']:
                all_rewards.append(step['Reward'])
                
        return {
            'mean': float(np.mean(all_rewards)),
            'std': float(np.std(all_rewards)),
            'min': float(np.min(all_rewards)),
            'max': float(np.max(all_rewards)),
            'distribution': np.histogram(all_rewards, bins=20)[0].tolist()
        }
    
    def analyze_state_coverage(self) -> Dict:
        """Analyze the distribution of states visited"""
        unique_states = set()
        state_transitions = defaultdict(int)
        
        for episode in self.eval_data:
            for i, step in enumerate(episode['Timesteps']):
                state = tuple(step['State'])
                unique_states.add(state)
                
                if i > 0:
                    prev_state = tuple(episode['Timesteps'][i-1]['State'])
                    state_transitions[(prev_state, state)] += 1
        
        return {
            'unique_states': len(unique_states),
            'total_transitions': sum(state_transitions.values()),
            'common_transitions': sorted(
                state_transitions.items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:10]
        }
    
    @staticmethod
    def _calculate_rolling_average(values: List[float], window: int = 10) -> List[float]:
        """Calculate rolling average with specified window size"""
        rolling_avg = []
        for i in range(len(values)):
            start_idx = max(0, i - window + 1)
            rolling_avg.append(np.mean(values[start_idx:i+1]))
        return rolling_avg

In [26]:
class ExtendedTrafficDashboard(TrafficDashboard):
    def __init__(self, training_data: Dict, evaluation_data: Dict):
        """Initialize dashboard with both training and evaluation data"""
        self.training_data = training_data
        self.evaluation_data = evaluation_data


    def create_dashboard(self) -> go.Figure:
        """Create the complete dashboard with training and evaluation comparisons"""
        # Create figure with subplots
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                'Training vs Evaluation Progress',
                'Action Distribution Comparison',
                'Lane Utilization',
                'Reward Distribution',
                'Episode Length Distribution',
                'State Coverage'
            ),
            specs=[[{}, {}],
                  [{}, {}],
                  [{}, {}]]
        )
        
        # Add all subplots
        self._add_training_eval_progress(fig, 1, 1)
        self._add_action_comparison(fig, 1, 2)
        self._add_lane_metrics(fig, 2, 1)
        self._add_reward_distribution(fig, 2, 2)
        self._add_episode_length_distribution(fig, 3, 1)
        self._add_state_coverage(fig, 3, 2)
        
        # Update overall layout
        fig.update_layout(
            height=1200,
            showlegend=True,
            title_text="Traffic Simulation Analysis - Training vs Evaluation",
            template='plotly_white'
        )
        
        return fig

    def _add_training_eval_progress(self, fig: go.Figure, row: int, col: int):
        """Add comparison of training and evaluation progress"""
        # Add training data
        fig.add_trace(
            go.Scatter(
                x=self.training_data['training_progress']['episodes'],
                y=self.training_data['training_progress']['rolling_avg'],
                name='Training Rolling Avg',
                line=dict(color='blue'),
                opacity=0.7
            ),
            row=row, col=col
        )
        
        # Add evaluation data
        fig.add_trace(
            go.Scatter(
                x=self.evaluation_data['episode_metrics']['episodes'],
                y=self.evaluation_data['episode_metrics']['rolling_avg_reward'],
                name='Evaluation Rolling Avg',
                line=dict(color='red'),
                opacity=0.7
            ),
            row=row, col=col
        )
        
        fig.update_xaxes(title_text="Episodes", row=row, col=col)
        fig.update_yaxes(title_text="Average Reward", row=row, col=col)

    def _add_reward_distribution(self, fig: go.Figure, row: int, col: int):
        """Compare reward distributions between training and evaluation with hover labels for means"""
        # Add training reward distribution
        fig.add_trace(
            go.Histogram(
                name='Training',
                x=self.training_data['value_stats']['distribution'],
                nbinsx=20,
                histnorm='percent',
                marker_color='blue',
                opacity=0.7
            ),
            row=row, col=col
        )
        
        # Add evaluation reward distribution
        fig.add_trace(
            go.Histogram(
                name='Evaluation',
                x=self.evaluation_data['reward_analysis']['distribution'],
                nbinsx=20,
                histnorm='percent',
                marker_color='red',
                opacity=0.7
            ),
            row=row, col=col
        )
        
        # Add vertical lines for means with hover text
        fig.add_trace(
            go.Scatter(
                x=[self.training_data['value_stats']['mean']],
                y=[0],  # Start at bottom of plot
                mode='lines',
                name='Training Mean',
                line=dict(color='blue', dash='dash'),
                hoverinfo='text',
                hovertext=f"Training Mean: {self.training_data['value_stats']['mean']:.2f}",
                showlegend=True
            ),
            row=row, col=col
        )
        
        fig.add_trace(
            go.Scatter(
                x=[self.evaluation_data['reward_analysis']['mean']],
                y=[0],  # Start at bottom of plot
                mode='lines',
                name='Evaluation Mean',
                line=dict(color='red', dash='dash'),
                hoverinfo='text',
                hovertext=f"Evaluation Mean: {self.evaluation_data['reward_analysis']['mean']:.2f}",
                showlegend=True
            ),
            row=row, col=col
        )
        
        # Add shapes for mean lines (extending full height of plot)
        fig.add_shape(
            type='line',
            x0=self.training_data['value_stats']['mean'],
            x1=self.training_data['value_stats']['mean'],
            y0=0,
            y1=1,
            yref='paper',
            line=dict(color='blue', dash='dash'),
            row=row, col=col
        )
        
        fig.add_shape(
            type='line',
            x0=self.evaluation_data['reward_analysis']['mean'],
            x1=self.evaluation_data['reward_analysis']['mean'],
            y0=0,
            y1=1,
            yref='paper',
            line=dict(color='red', dash='dash'),
            row=row, col=col
        )
        
        # Update axes
        fig.update_xaxes(title_text="Reward Value", row=row, col=col)
        fig.update_yaxes(title_text="Frequency (%)", row=row, col=col) 


        
    def _add_action_comparison(self, fig: go.Figure, row: int, col: int):
        """Compare action distributions between training and evaluation"""
        # Prepare data
        actions = ['Left', 'Stay', 'Right']
        training_pcts = [d['percentage'] for d in self.training_data['action_distribution']]
        eval_pcts = [d['percentage'] for d in self.evaluation_data['action_distribution']]
        
        # Add training data bars
        fig.add_trace(
            go.Bar(
                name='Training',
                x=actions,
                y=training_pcts,
                marker_color='blue',
                opacity=0.7,
                width=0.3,
                text=[f'{x:.1f}%' for x in training_pcts],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        # Add evaluation data bars
        fig.add_trace(
            go.Bar(
                name='Evaluation',
                x=actions,
                y=eval_pcts,
                marker_color='red',
                opacity=0.7,
                width=0.3,
                text=[f'{x:.1f}%' for x in eval_pcts],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        # Update layout
        fig.update_xaxes(
            title_text="Action Type",
            categoryorder='array',
            categoryarray=actions,
            row=row, col=col
        )
        fig.update_yaxes(
            title_text="Selection Frequency (%)",
            range=[0, 100],
            row=row, col=col
        )

    def _add_lane_metrics(self, fig: go.Figure, row: int, col: int):
        """Compare lane utilization and performance metrics"""
        # Prepare data
        training_lanes = {d['lane']: d for d in self.training_data['lane_metrics']}
        eval_lanes = {d['lane']: d for d in self.evaluation_data['lane_transitions']}
        lanes = sorted(set(training_lanes.keys()) | set(eval_lanes.keys()))
        
        # Create bar traces for visits
        fig.add_trace(
            go.Bar(
                name='Training Visits',
                x=lanes,
                y=[training_lanes.get(lane, {'visits': 0})['visits'] for lane in lanes],
                marker_color='blue',
                opacity=0.7,
                text=[training_lanes.get(lane, {'visits': 0})['visits'] for lane in lanes],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        fig.add_trace(
            go.Bar(
                name='Evaluation Visits',
                x=lanes,
                y=[eval_lanes.get(lane, {'visits': 0})['visits'] for lane in lanes],
                marker_color='red',
                opacity=0.7,
                text=[eval_lanes.get(lane, {'visits': 0})['visits'] for lane in lanes],
                textposition='auto',
            ),
            row=row, col=col
        )
        
        # Add line traces for average rewards/values
        fig.add_trace(
            go.Scatter(
                name='Training Avg Value',
                x=lanes,
                y=[training_lanes.get(lane, {'avg_value': 0})['avg_value'] for lane in lanes],
                mode='lines+markers',
                line=dict(color='darkblue', dash='dash'),
                yaxis='y2'
            ),
            row=row, col=col
        )
        
        fig.add_trace(
            go.Scatter(
                name='Evaluation Avg Reward',
                x=lanes,
                y=[eval_lanes.get(lane, {'avg_reward': 0})['avg_reward'] for lane in lanes],
                mode='lines+markers',
                line=dict(color='darkred', dash='dash'),
                yaxis='y2'
            ),
            row=row, col=col
        )
        
        # Update axes
        fig.update_xaxes(title_text="Lane Number", row=row, col=col)
        fig.update_yaxes(title_text="Visit Count", row=row, col=col)
        fig.update_yaxes(title_text="Average Value/Reward", secondary_y=True, row=row, col=col)




    def _add_episode_length_distribution(self, fig: go.Figure, row: int, col: int):
        """Visualize distribution of episode lengths"""
        # Get episode lengths
        eval_lengths = self.evaluation_data['episode_metrics']['steps']
        
        # Create histogram
        fig.add_trace(
            go.Histogram(
                x=eval_lengths,
                nbinsx=20,
                name='Episode Lengths',
                marker_color='purple',
                opacity=0.7
            ),
            row=row, col=col
        )
        
        # Add mean line
        mean_length = np.mean(eval_lengths)
        fig.add_vline(
            x=mean_length,
            line_dash="dash",
            line_color="red",
            annotation_text=f"Mean: {mean_length:.1f} steps",
            row=row, col=col
        )
        
        # Update axes
        fig.update_xaxes(title_text="Episode Length (steps)", row=row, col=col)
        fig.update_yaxes(title_text="Frequency", row=row, col=col)

    def _add_state_coverage(self, fig: go.Figure, row: int, col: int):
        """Visualize state coverage metrics"""
        # Get state coverage data
        coverage = self.evaluation_data['state_coverage']
        
        # Create a summary box
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=0.5,
            text=f"""
            <b>State Coverage Analysis</b><br><br>
            Unique States Visited: {coverage['unique_states']}<br>
            Total Transitions: {coverage['total_transitions']}<br><br>
            Top State Transitions:<br>
            {self._format_top_transitions(coverage['common_transitions'][:5])}
            """,
            showarrow=False,
            font=dict(size=12),
            align="center",
            bordercolor="black",
            borderwidth=1,
            borderpad=4,
            bgcolor="white",
            row=row,
            col=col
        )
        
        # Update axes (empty in this case)
        fig.update_xaxes(showticklabels=False, showgrid=False, row=row, col=col)
        fig.update_yaxes(showticklabels=False, showgrid=False, row=row, col=col)

    @staticmethod
    def _format_top_transitions(transitions):
        """Helper method to format transition data for display"""
        formatted = []
        for (state1, state2), count in transitions:
            formatted.append(f"Count: {count}")
        return "<br>".join(formatted)

In [34]:
# Initialize processors
training_processor = TrafficDataProcessor('./TD_Task1/Weighted_model.json')
eval_processor = EvaluationDataProcessor('./TD_Task1/Weighted_model_test.json')

# Get data for dashboard
training_data = training_processor.get_dashboard_data()
eval_data = eval_processor.get_evaluation_metrics()

# Create and display dashboard
dashboard = ExtendedTrafficDashboard(training_data, eval_data)
fig = dashboard.create_dashboard()
fig.update_layout(
    height=1500,  # Increased height for better visibility
    title=dict(
        text="Traffic Simulation Analysis - Training vs Evaluation",
        x=0.5,
        y=0.95,
        xanchor='center',
        font=dict(size=20)
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255,255,255,0.8)"
    ),
    margin=dict(t=100, b=50, l=50, r=50)
)

fig.show()
